<a href="https://colab.research.google.com/github/Gibbons-Lab/isb_course_2023/blob/main/16S_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦠 Amplicon Sequencing Data Analysis with QIIME 2

This notebook will accompany the first session of the 2024 ISB Virtual Microbiome Series. The presentation slides can be [found here](https://gibbons-lab.github.io/isb_course_2024/16S).

Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be prompted to trust the notebook. We promise that it is safe 🤞

**Disclaimer:**

The Google Colab notebook environment will interpret any command as Python code by default. If we want to run bash commands we will have to prefix them by `!`. So any command you see with a leading `!` is a bash command and if you wanted to run it in your terminal you would omit the `!`. For example, if in the Colab notebook you ran `!wget` you would just run `wget` in your terminal.

**Run all cells IN ORDER**

## Setup

QIIME 2 is usually installed by following the [official installation instructions](https://docs.qiime2.org/2024.5/install/). However, because we are using Google Colab and there are some caveats to using conda here, we will have to hack around the installation a little bit. But no worries, we provide a setup script below which does all this work for us. 😌

So...let's start by pulling a local copy of the project repository down from GitHub.

In [ ]:
!git clone https://github.com/gibbons-lab/isb_course_2024 materials

This repository, called __materials__, contains all the relevant data and other resources we'll need for this course. To view the directory, click on the folder icon on the left. Let's navigate to that directory via command line now:

In [ ]:
%cd materials

Notice here we use ```%``` instead of ```!``` to run out command line function. This makes the path change to our directory permanent: using the ```!``` operator only switches the interpreter to expect command line prompts temporarily.




## Install QIIME2

Now that we have all our materials, we're _almost_ ready to get started - but not quite. Remember QIIME2? We'll need to install that before getting into the actual analysis. Don't worry - this will only set up in the Colab notebook, not on your local machine.

Let's run the following cell, to install and setup QIIME2.

In [ ]:
%run setup_qiime2

⬆️ This will take some time (usually 10 to 15 minutes), so we'll switch back over to the [presentation](https://gibbons-lab.github.io/isb_course_2024/16S) while we wait.

If you want to learn more about QIIME2, we recommend you check out the [documentation](https://docs.qiime2.org/). This will also explain how to install QIIME2 on your local machine 🖥

## Let's Get Started!

Now we're on to the fun part. Let's begin by taking a look at our data. In the __data__ folder, you'll find 10 FASTQ files, a file manifest, and a metadata file. Let's take a look at the manifest, first. This is a file that contains the name, and filepath of all our samples, and we'll need it later on when we use QIIME2 📝.

In [ ]:
import pandas as pd
manifest = pd.read_csv('data/manifest.tsv', sep = '\t')
manifest

We can also check out the metadata file, which will give us more context on our samples 🔬

In [ ]:
metadata = pd.read_csv('data/metadata.tsv', sep='\t')
metadata

Looks good, all 10 FASTQ files are accounted for, five healthy and five with Parkinson's Disease. We can use the manifest file to import our files into QIIME2.

## QIIME2 Pipeline

Let's remind ourselves what the QIIME2 pipeline will do:
![our workflow](https://github.com/Gibbons-Lab/isb_course_2024/raw/main/docs/16S/assets/steps.png)

To use sequencing data in QIIME2, we first need to turn the FASTQ files containing our data into QIIME artifacts. Using the manifest we just checked out, let's run our first command:

-- as a reminder, adding ```!``` before the command tells the notebook this is a bash command, rather than python.

In [ ]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path data/manifest.tsv \
  --output-path sequences.qza \
  --input-format SingleEndFastqManifestPhred33V2

Let's take a look a the command. QIIME commands take following format:

```
qiime plugin action --i-argument1 ... --o-argument2 ...
```
In the previous command, we are calling the ```tools``` plugin within QIIME2 to import our data. The following arguments designate where the manifest is, as well as where the output should be saved. We also tell QIIME2 what sort of input to expect.

Argument types usually begin with a letter denoting their meaning:

- `--i-...` = input files
- `--o-...` = output files
- `--p-...` = parameters
- `--m-...` = metadata

---

## Visualizing our Data 🔎

Before we move on, let's use QIIME2 to visualize our sequencing data.

In [ ]:
!qiime demux summarize \
--i-data sequences.qza \
--o-visualization qualities.qzv

.qzv files like the one we just produced are visualization. You can view the plot by downloading the file and opening it using http://view.qiime2.org. To download the file click on the folder symbol to the left, open the `materials` folder, and choose download from the dot menu next to the `qualities.qzv` file.

---

## Quality Filtering


Before we can use our sequencing data, we need to "denoise" it. To do this, we'll use a plugin called DADA2. This involves three things.

1. filter and trim the reads
2. find the most likely set of unique sequences in the sample (ASVs)
3. remove chimeras
4. count the abundances of each ASV


This command will take a little time - let's run it, and head back to the presentation to discuss what's happening.

In [ ]:
!qiime dada2 denoise-single \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len 150 \
    --p-n-threads 2 \
    --output-dir dada --verbose


If this step takes too long or fails, you can also copy the results from the treasure chest with the following command.

In [ ]:
# obscure magic that will only copy if the previous command failed
![ -d dada ] || cp -r treasure_chest/dada .

Let's check to see how that went. One good way to tell if the identified ASVs are representative of the sample is to see how many reads were maintained throughout the pipeline. Here, the most common issues and solutions are:

**Large fraction of reads is lost during merging (only paired-end)**

![read overlap](https://gibbons-lab.github.io/isb_course_2024/16S/assets/read_overlap.png)

In order to merge ASVs DADA2 uses an overlap of 12 bases between forward and reverse reads by default. Thus, your reads must allow for sufficient overlap *after* trimming. So if your amplified region is 450bp long and you have 2x250bp reads and you trim the last 30 bases of each read, truncating the length to 220bp, the total length of covered sequence is 2x220 = 440 which is shorter than 450bp so there will be no overlap. To solve this issue trim less of the reads or adjust the `--p-min-overlap` parameters to something lower (but not too low).

<br>

**Most of the reads are lost as chimeric**

![read overlap](https://gibbons-lab.github.io/isb_course_2024/16S/assets/chimera.png)

This is usually an experimental issue as chimeras are introduced during amplification. If you can adjust your PCR, try to run fewer cycles. Chimeras can also be introduced by incorrect merging. If your minimum overlap is too small ASVs may be merged randomly. Possible fixes are to increase the `--p-min-overlap` parameter or run the analysis on the forward reads only (in our empirical observations, chimeras are more likely to be introduced in the joined reads). *However, losing between 5-25% of your reads to chimeras is normal and does not require any adjustments.*


Our denoising stats are contained in an artifact. To convert it to a visualization we can use `qiime metadata tabulate`.

In [ ]:
!qiime metadata tabulate \
    --m-input-file dada/denoising_stats.qza \
    --o-visualization dada/denoising-stats.qzv

Like before, we can download the .qzv file and visualize the results using the [QIIME2 Viewer]('https://view.qiime2.org/').

It's important to understand what this output tells us. For instance, what percent of reads in our data pass the filtering step? What percent of reads were non-chimeric? Differences in these metrics between samples can affect diversity metrics.

---

## Diversity and Phylogenetics

### Introduction to diversity metrics

An important metric to consider when studying microbial ecology is __diversity__. Diversity comes in two flavors: ⍺ (alpha) and β (beta).

Alpha diversity is pretty simple - how diverse is a single sample? You might consider measures like richness and evenness.

![alpha diversity](https://gibbons-lab.github.io/isb_course_2024/16S/assets/alpha_diversity.png)

Beta diversity instead looks at how different two samples are from each other - what taxa are shared, and how their abundances differ.

![beta diversity](https://gibbons-lab.github.io/isb_course_2024/16S/assets/beta_diversity.png)

I want to note here that we're getting into some analyses that some bioinformaticians may prefer to do in R or Python, outside of the QIIME framework. For example, diversity analyses can be performed using `scikit-bio` in Python or `vegan` in R. In fact, QIIME 2 is using `scikit-bio` and `vegan` under the hood for its diversity calculations! QIIME eliminates the need to manipulate dataframes or do calculations yourself, but it might not have the newest methods!

### Starting our Tree

Let's start by building a phylogenetic tree for our sequences using the following command. This time, we call the _phylogeny_ plugin in QIIME2.

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada/representative_sequences.qza \
    --output-dir tree

We can create a visualization for the tree using the [empress](https://github.com/biocore/empress) QIIME 2 plugin.

In [ ]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

## Calculating Diversity

Using the Diversity plugin, we can use our table and tree to calculate several diversity metrics. To account for variations in sampling depth, we'll provide QIIME2 with a cutoff at which rarefy all our samples. Since this randomly selects sequences, your results might look a little different. We'll also pass in our metadata file, so we can keep track how which samples come from each group.

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table dada/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file data/metadata.tsv \
    --output-dir diversity

If you open the `diversity` folder, you'll see that we calculated several different diversity metrics. Beta diversity uses a "distance" or "dissimilarity" matrix, but there are different definitions of distance! Some different types of distance include Bray-Curtis, Jaccard, Unweighted UniFrac, and weighted UniFrac. UniFrac distances are based on phylogeny, while Bray-Curtis and Jaccard are not. For more information on diversity metrics, check out this QIIME [forum post](https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282).

## Alpha Diversity

We get a bunch of outputs from the previous command - measures of both alpha and beta diversity. To start, let's use the Shannon vector in the output directory to create a visualization of alpha diversity across samples. Generally, healthy, long-living individuals have balanced diverse microbiomes. However, this isn't necessarily a direct indicator of health or disease. Let's see how it looks in our samples

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization diversity/alpha_groups.qzv

Like before, we can download the visualization and open it with the QIIME2 viewer.

There doesn't appear to be a difference in Shannon Diversity between Parkinson's Disease patients and healthy controls, but could there be confounding variables? Since this is a cohort study rather than a controlled experiment, we can't control for variables that affect microbiome composition and diversity like antibiotics, diet, other medications, ..., but we can __stratify__ across them. Let's see what happens if we stratify by drug use.

In [ ]:
!

## Beta Diversity

Let's visualize the beta diversity and see how they separate. For this we'll look at weighted UniFrac. This time, we'll have to download the file ⬅️

We can check for 'significant' separation between samples using PERMANOVA. We can do this with the diversity plugin in QIIME2.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "parkinson_disease" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

We can also use PERMANOVA to identify confounders. PERMANOVA tells us how much variance in the community composition is explained by each variable. Common confounders include sex, age, BMI, diet, and antibiotic use. In the original study, authors identified that Parkinson's Disease medications were associated with different microbiome compositions. Let's take a look at these variables.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "parkinson_disease + sex + age + location + p3m_antibiotics_bool" \
    --p-n-jobs 10 \
    --o-visualization diversity/permanova_big.qzv

Before, we did not see a significant p-value for the effect of Parkinson's disease on beta diversity. However, when we add certain covariates, we might find that they were confounding a relationship.

However, most of our variance remains unexplained. Microbiome composition is affected by many things, and this is an uncontrolled cohort study, so we would not expect any single variable to explain most of the variance.

## Visualizing Beta Diversity Using PCoA

If we want to __visually__ show this separation between samples, we can't just plot the entire UniFrac distance matrix, because it has 100+ dimensions! Instead, we can use **dimensionality reduction** to "compress" our data into a few dimensions that explain most of the variance. There are several types of dimensionality reduction (like UMAP and tSNE), but the preferred method of dimensionality reduction for microbiome communities is Principal Coordinate Analysis (PCoA). This is because PCoA is linear, and thus preserves the global structure of the data and is reproducible.

We already ran a PCoAs for each distance metric, and we can look at them if we download `weighted_unifrac_emperor.qzv`, `unweighted_unifrac_emperor.qzv`, `bray_curtis_emperor.qzv`, or `jaccard_emperor.qzv`

---
## Taxonomic Classification

We can learn a lot from diversity metrics, alpha and beta. But to really dig into the data, we need to know what microbes are in each sample 🦠. To do this, we'll classify the reads in QIIME2 using a Bayesian classifier. Several such classifiers are available at https://docs.qiime2.org/2024.5/data-resources/

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads dada/representative_sequences.qza \
    --i-classifier ncbi-refseq-genus-515f-806r.qza \
    --p-n-jobs 2 \
    --o-classification taxa.qza

Now we've classified the reads, we can visualize the taxonomic breakdown of our samples.

In [ ]:
!qiime taxa barplot \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization taxa_barplot.qzv

Now, we can use ```table.qza```, which contains our reads, and ```taxa.qza```, which contains taxonomic classifications for reads, and collapse the data onto the genus level.

In [ ]:
!qiime taxa collapse \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --p-level 6 \
    --o-collapsed-table genus.qza

We'll export this as a .tsv, which will be more usable for the next portion of the course that you'll see tomorrow

In [ ]:
!qiime tools export \
    --input-path genus.qza \
    --output-path exported
!biom convert -i exported/feature-table.biom -o genus.tsv --to-tsv

Let's peek at the results 🔭

In [ ]:
abundances = pd.read_table("genus.tsv", skiprows=1, index_col=0)
abundances

This is easier to interpret by visualizing the results. We can use the file we just exported from QIIME2 to build a visualization using any tool we like, such as seaborn or plotnine. Here is an example of building a visualization (a heatmap) in seaborn:

In [ ]:
import numpy as np
import seaborn as sns

In [ ]:
abund_to_plot = abundances
abund_to_plot.index = abund_to_plot.index.str.split(";").str[5]          # Use only the genus name
abund_to_plot = abund_to_plot[~abund_to_plot.index.isin(["g__", "__"])]  # remove unclassified genera
abund_to_plot = abund_to_plot.sample(50, axis=0)                         # use 50 random genera (rows)

# Let's do a centered log-ratio transform: log x_i - log mean(x)
transformed = abund_to_plot.apply(
    lambda xs: np.log(xs + 0.5) - np.log(xs.mean() + 0.5),
    axis=0)

sns.clustermap(transformed.T, cmap="magma", xticklabels=True, figsize=(16, 6))

Now, our data is starting to be interpretable. Each row is a sample, and each column is a bacterial genus. The table values are "counts", or the number of times a genus was detected in a certain sample. We can use relative abundance data to test hypotheses, but it requires special statistical methods because it is ___compositional___

## Differential Abundance Analysis

Explain compositional data + methods for testing

In [ ]:
!qiime composition ancombc \
  --i-table genus.qza \
  --m-metadata-file data/metadata.tsv \
  --p-formula "parkinson_disease" \
  --o-differentials ancombc.qza

In [ ]:
!qiime composition da-barplot \
  --i-data ancombc.qza \
  --p-significance-threshold 0.01 \
  --o-visualization da_barplot.qzv

## Exercise - Plant a Tree

One visualization that we did not spend a lot of time on was the phylogentic tree of our ASVs. Let's change that! We have seen that there are genera that appear in multiple populations in the previous step. But are the organisms in that genus actually the same?

Let's annotate the tree with our taxonomic classifications and abundances. We will use the empress plugin again but this time with the `community-plot` option. I filled in a template of the command for you. Can you figure out what has to go in the empty spaces?

**QUESTIONS:**

1) Are some of the branch lengths on the tree longer than you would expect? Do you notice anything interesting or suspicious about the taxonomic identities of these branches?

2) Can you find examples of phyla that are polyphyletic (i.e. where clusters of ASVs from the same phylum are found in different locations on the tree, showing different commmon ancestors)? What about polyphyletic taxa at lower taxonomic levels, like at the family or genus levels? Why do you think these patterns exist?

In [ ]:
# This won't run until you fill in the [EMPTY] spots with the right files ;)

!qiime empress community-plot \
    --i-tree [EMPTY] \
    --i-feature-table dada/table.qza \
    --m-sample-metadata-file [EMPTY] \
    --m-feature-metadata-file taxa.qza \
    --o-visualization community-tree-viz.qzv